# LSTM Encoder

RNN model to encode an abstract and predict the gender of the study

In [1]:
%store -r abstracts_w2v_embeddings abstracts_targets_collapsed

### Training Data

In [2]:
from sklearn.cross_validation import train_test_split

X_idxs = list(abstracts_targets_collapsed.abstracts_idxed)
ys = abstracts_targets_collapsed.gender_binarized
X_train, X_val, ys_train, ys_val = train_test_split(X_idxs, ys)
num_classes = len(ys.unique())

### Train

In [ ]:
from rnn.train import RNNTrainer

trainer = RNNTrainer(abstracts_w2v_embeddings, num_classes=num_classes)

trainer.train(X_train, ys_train, X_val, ys_val, val_every=100)

pd.Series(trainer.val_loss).plot()

Building theano graph...
Done!
Validation Loss: 1.3837204964
Validation Loss: 0.963892744162
Validation Loss: 0.803409818786
Validation Loss: 0.702164951711
Validation Loss: 0.648593164271
Validation Loss: 0.663617110873
Validation Loss: 0.657808526296
Validation Loss: 0.589535977047
Validation Loss: 0.621121454755
Validation Loss: 0.604897697098